In [2]:
import numpy as np
import random as rand

**Initialize Network**

In [24]:
Network = [3, 5, 1]

In [25]:
x = 1-2*np.random.rand(1,Network[0])
w1 = 1-2*np.random.rand(Network[0],Network[1])
w2 = 1-2*np.random.rand(Network[1],Network[2])
b1 = 1-2*np.random.rand(1,Network[1])
b2 = 1-2*np.random.rand(1,Network[2])

In [26]:
def sig(inputs,weights, bias):
    linCom = np.dot(inputs,weights)-bias
    return 1/(1+np.exp(-linCom))

**Generate Data**

In [11]:
x = 1 - 2*np.random.rand(1,3)
if x[0][2]>0:
    y = np.array([1,0])
else:
    y = np.array([0,1])
z = np.array([x,y])
print(z)

[array([[-0.16784681,  0.7547066 ,  0.63286029]]) array([1, 0])]


In [13]:
x = np.array([1,2])
y = np.array([3,4])

In [25]:
z=np.append([x],[y],axis=0)
z

array([[1, 2],
       [3, 4]])

In [34]:
z=np.append(z,[[5,6]],axis=0)
z

array([[1, 2],
       [3, 4],
       [5, 6],
       [5, 6],
       [5, 6],
       [5, 6],
       [5, 6]])